In [1]:
from openai import api_key
from scipy.stats import cosine

print("OK")

OK


In [2]:
%pwd

'c:\\Coding\\Projects\\Medical-chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Coding\\Projects\\Medical-chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader

c:\Users\joabd\miniconda3\envs\medibot\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\joabd\miniconda3\envs\medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Extract text from PDF files
def extract_text_from_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    return documents

In [7]:
extracted_data=extract_text_from_pdf(r"C:\Coding\Projects\Medical-chatbot\data\Medical_book.pdf")

In [49]:
extracted_data[:2]

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')]

In [9]:
len(extracted_data)

637

In [10]:
from typing import List
from langchain.schema import Document

In [11]:
def clean_metadata(docs:List[Document])->List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects containing
    only 'source' in metadata and the original page_content
    :param docs: Original list of Documents with a lot of unwanted metadata information
    :return: Cleaned list of Documents with only necessary metadata and the original page_content
    """
    cleaned_doc: List[Document]=[]
    for doc in docs:
        src=doc.metadata.get("source")
        cleaned_doc.append(
            Document(
                page_content=doc.page_content,
                metadata={"source":src}
            )
        )
    return cleaned_doc

In [12]:
cleaned_document=clean_metadata(extracted_data)

In [50]:
cleaned_document[:2]

[Document(metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content=''),
 Document(metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')]

In [52]:
len(cleaned_document)

637

In [53]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [54]:
# Split the cleaned documents into smaller chunks
def document_into_chunks(cleaned_documents):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts=text_splitter.split_documents(cleaned_documents)
    return texts

In [55]:
chunks=document_into_chunks(cleaned_document)

In [56]:
chunks[:2]

[Document(metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1')]

In [18]:
len(chunks)

5859

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings

In [20]:
# Downloading the embedding model
def download_embeddings():
    """
    Download and return the HuggingFace Embedding model
    :return: HuggingFace embedding model
    """
    model='sentence-transformers/all-MiniLM-L6-v2'
    embeddings=HuggingFaceEmbeddings(
        model_name=model
    )
    return embeddings


In [21]:
embedding_model=download_embeddings()

C:\Users\joabd\AppData\Local\Temp\ipykernel_29780\788224968.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(


In [22]:
embedding_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [44]:
vector=embedding_model.embed_query("Hello world")
vector[:10]

[-0.03447723016142845,
 0.031023241579532623,
 0.006735020782798529,
 0.026109028607606888,
 -0.039362046867609024,
 -0.16030247509479523,
 0.06692402064800262,
 -0.006441468372941017,
 -0.04745054990053177,
 0.01475888304412365]

In [24]:
# Embedding model is a sentence transformer so it converts a sentence into a 384 dimensional vector
len(vector)

384

In [25]:
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [27]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [28]:
from pinecone import Pinecone
pc=Pinecone(api_key=PINECONE_API_KEY)

In [29]:
pc

In [30]:
from pinecone import ServerlessSpec
index_name="medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384, # Dimensionality of the embedding
        metric="cosine", # Cosine Similarity metric
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index=pc.Index(index_name)

#### **Note:**
- We do not convert the chunks into vectors and store the vectors in the pinecone vector database
- We just give the chunks, the embedding model and the index name, pinecone converts the chunks to the vectors and vice versa using the embedding model we specified.
- <span style="background-color:orange;">When you upload the documents to the pinecone vector database, it does not check if the documents are already present in a given index so there is high possibility of duplication of documents in case you run the code cell twice.</span>

In [31]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embedding_model,
    index_name=index_name
)

In [32]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [33]:
retrieved_docs=retriever.invoke("What is acne?")

In [34]:
retrieved_docs

[Document(id='65b852ad-351e-488f-b122-8eb48c662817', metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='60e4aa36-e1a6-40f5-aa0d-1395943ebf0b', metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='d76149d0-703f-4271-8186-667327103af9', metadata={'source': 'C:\\Coding\\Projects\\Medical-chatbot\\data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26')]

In [35]:
from langchain_openai import ChatOpenAI

ChatModel=ChatOpenAI(model="gpt-4o")

In [36]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [37]:
system_prompt=(
    "You are a medical assistant for question-answering tasks. "
    "Use the following piece of retrieved context to answer "
    "the question. If you do not know the answer, say that you "
    "do not know. Use three sentences at max and keep the answer concise. "
    "\n\n"
    "{context}"
)

In [38]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [39]:
question_answer_chain=create_stuff_documents_chain(ChatModel,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [40]:
response=rag_chain.invoke({"input":"what is Acromegaly and Gigantism ?"})
print(response["answer"])

Acromegaly is a disorder characterized by the abnormal release of a chemical from the pituitary gland, leading to increased growth in bones and soft tissue and causing various disturbances throughout the body. Gigantism refers to similar excessive growth but occurs in children or adolescents whose growth plates have not yet closed. Both conditions are related to hormonal imbalances involving the pituitary gland.


In [41]:
response=rag_chain.invoke({"input":"what is acne ?"})
print(response["answer"])

Acne is a skin condition characterized by the appearance of pimples, blackheads, whiteheads, cysts, and nodules due to clogged hair follicles and sebaceous (oil) glands. It often affects teenagers but can occur at any age. It can appear on the face, back, chest, and shoulders.


#### **The following cells show us the model working without conversational memory:**
- The model is not able to correctly answer questions related to the `treatment of acne`, as this question was asked indirectly not explicitly as the other questions asked before.

In [42]:
response=rag_chain.invoke({"input":"what is the best treatment for it ?"})
print(response["answer"])

The best treatment approach depends on the severity of the case. In severe cases of deformity, surgery may be performed to straighten and realign the spine or replace joints. For managing inflammation and pain, alternative treatments like herbal remedies, acupuncture, and homeopathic remedies may be helpful.


In [43]:
response=rag_chain.invoke({"input":"what are we talking about right now ?"})
print(response["answer"])

The retrieved context is about bites and stings.
